In [51]:
import pandas as pd
import os
import re

rootdir = "/home/arav/Unessay/"

regex_demo = re.compile('.*Demo.*')
regex_immu = re.compile('.*Immu.*')

list_of_demo_names = []
list_of_immu_names = []

for root, dirs, files in os.walk(rootdir):
  for file in files:
    if regex_demo.match(file):
      list_of_demo_names.append(file)
list_of_demo_names.sort()

for root, dirs, files in os.walk(rootdir):
  for file in files:
    if regex_immu.match(file):
      list_of_immu_names.append(file)
list_of_immu_names.sort()
 
  
# df_99_00_demo = pd.read_csv("./1999_2000_Demo_NHANES.csv").loc[:, ['SEQN', ]]

In [77]:
df_99_00_demo = pd.read_csv("2009-2010_Immu_NHANES.csv").loc[:, ['SEQN', 'IMQ020']]
df_99_00_demo = df_99_00_demo.loc[df_99_00_demo["IMQ020"] == "."]
df_99_00_demo

,SEQN,IMQ020


In [52]:
col_names_across_demo_data = list()
for i in range(0,len(list_of_demo_names)-1):
  first_df = list(pd.read_csv("./" + list_of_demo_names[i]).columns.values)
  second_df = list(pd.read_csv("./" + list_of_demo_names[i+1]).columns.values)
  col_names_across_demo_data += first_df
  col_names_across_demo_data += second_df
demo_col_count = pd.Series(col_names_across_demo_data).value_counts()

In [53]:
col_names_across_immu_data = list()
for i in range(0,len(list_of_immu_names)-1):
  first_df = list(pd.read_csv("./" + list_of_immu_names[i]).columns.values)
  second_df = list(pd.read_csv("./" + list_of_immu_names[i+1]).columns.values)
  col_names_across_immu_data += first_df
  col_names_across_immu_data += second_df
immu_col_count = pd.Series(col_names_across_immu_data).value_counts()

In [54]:
# Check that all SEQN numbers are unique across demographic and immunization data
SEQN_list = []
for i in range(0, len(list_of_demo_names)):
  df = list(pd.read_csv("./" + list_of_demo_names[i]).SEQN)
  SEQN_list += df
  
for i in range(0, len(list_of_immu_names)):
  df = list(pd.read_csv("./" + list_of_immu_names[i]).SEQN)
  SEQN_list += df

SEQN_list.sort()
SEQN_unique_sanity_check = pd.Series(SEQN_list).value_counts(ascending=True)
SEQN_unique_sanity_check

# This confirms that each participant number is used twice in both the demographic and immunization data

1.0         2
71755.0     2
71754.0     2
71753.0     2
71752.0     2
           ..
35869.0     2
35868.0     2
35867.0     2
35891.0     2
124822.0    2
Length: 107622, dtype: int64

In [83]:
# immu_data
# for i in range(0, len(list_of_immu_names)):
#   immu_data_list.append(pd.read_csv("./" + list_of_immu_names[i]).loc[:,['SEQN','IMQ020']])
years = ["1999-2000", "2001-2002", "2003-2004", "2005-2006", "2007-2008", "2009-2010", "2011-2012", "2013-2014", "2015-2016", "2017-2020"]

percent_vaccinated_all = pd.DataFrame()

for i in range(0, len(list_of_immu_names)):
  immu_data = pd.read_csv("./" + list_of_immu_names[i]).loc[:,["SEQN","IMQ020"]].set_index("SEQN").dropna(subset=['IMQ020'])
  
  immu_data.IMQ020 = immu_data.IMQ020.map(lambda p: 1 if (p == 1 or p == 2) else 0)
  
  demo_data = pd.read_csv("./" + list_of_demo_names[i]).loc[:, ["SEQN", "RIDRETH1"]].set_index("SEQN")
  grouped_data = immu_data.join(demo_data).rename(columns={"IMQ020":"Received At Least One Dose of Hepatitis B Vaccine","RIDRETH1":"Race"})
  percent_vacc = (grouped_data.groupby(["Race"]).sum()/grouped_data.groupby(["Race"]).count()).reset_index()
  percent_vacc["Year"] = years[i]
  
  percent_vaccinated_all = pd.concat([percent_vaccinated_all, percent_vacc])
  

# immu_99_00 = pd.read_csv("./1999_2000_Immu_NHANES.csv").loc[:,["SEQN","IMQ020"]].set_index("SEQN")
# immu_99_00.IMQ020 = immu_99_00.IMQ020.map(lambda p: 1 if (p == 1 or p == 2) else 0)

# demo_99_00 = pd.read_csv("./1999_2000_Demo_NHANES.csv").loc[:, ["SEQN", "RIDRETH1"]].set_index("SEQN")

# grouped_99_00 = immu_99_00.join(demo_99_00).rename(columns={"IMQ020":"Received At Least One Dose of Hepatitis B Vaccine","RIDRETH1":"Race"})
# # grouped_99_00 = grouped_99_00.groupby(["Race"])["Received At Least One Dose of Hepatitis B Vaccine"].sum() / 
# percent_vacc_99_00 = grouped_99_00.groupby(["Race"]).sum()/grouped_99_00.groupby(["Race"]).count()
# percent_vacc_99_00["Year"] = years[i]

# percent_vaccined_all = pd.concat([percent_vaccined_all, percent_vacc])

0    5903
1    4062
Name: Received At Least One Dose of Hepatitis B Vaccine, dtype: int64
0    5942
1    5097
Name: Received At Least One Dose of Hepatitis B Vaccine, dtype: int64
1    5204
0    4918
Name: Received At Least One Dose of Hepatitis B Vaccine, dtype: int64
1    5831
0    4517
Name: Received At Least One Dose of Hepatitis B Vaccine, dtype: int64
0    5121
1    5028
Name: Received At Least One Dose of Hepatitis B Vaccine, dtype: int64
1    5372
0    5165
Name: Received At Least One Dose of Hepatitis B Vaccine, dtype: int64
1    5288
0    4468
Name: Received At Least One Dose of Hepatitis B Vaccine, dtype: int64
1    5683
0    4492
Name: Received At Least One Dose of Hepatitis B Vaccine, dtype: int64
1    4991
0    4980
Name: Received At Least One Dose of Hepatitis B Vaccine, dtype: int64
0    8012
1    7548
Name: Received At Least One Dose of Hepatitis B Vaccine, dtype: int64


In [66]:
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

fig = px.line(percent_vaccinated_all,
              x="Year",
              y="Received At Least One Dose of Hepatitis B Vaccine",
              color="Race",
              title="Percent of Each Race Vaccinated for Hepatitis B"
              )
newnames = {"1.0":"Mexican American",
                "2.0":"Other Hispanic",
                "3.0":"Non-Hispanic White",
                "4.0":"Non-Hispanic Black",
                "5.0":"Other Race - Including Multi-Racial"}
fig = fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                    )
                        )
fig.write_html("./HTML_Graphs/Percent_Vaccinated_By_Race_Over_Time.html")
fig.show()